# Drawing Generator

This notebook implements utilities for generating "fake" architectural drawings.  It does so using examples of drawing icons that have been annotated with bounding boxes.

To do:
* Create a corpus of drawings in the right format for ingestion by object detection networks
* Maybe have a flag that keeps icons from overlapping in the drawing, but need to be careful that we may get into a situation where it is impossible to add more icons without overlap
* Set foreground and background colors of icons and drawings?

This note book creates image files with the specified objects. Randomly the object location is identified. If it is not a free location then all the locations in eight directions are examined till the boundary of image. If no locations are found with a prior fixed random generations then a sequential search is made to locate free location. The above steps ensure the required number of objects in the image. Still if the count of objects are not the same as specified, the image is rejected and a new image is created in its place, the above step is also tried for a fixed number of times. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random
import os
from collections import defaultdict
import os.path
import cv2
from numpy import zeros, newaxis

## Load annotations

In [2]:
# Path to the folder containing annotations
ANNO_DIR = '/home/sage08ai/Desktop/CV_tasks/door/symbols/'

In [3]:
def load_annotations(anno_dir = ANNO_DIR, labels = None, verbose = False):
    """
    Load all annotations found in a directory tree.  It is assumed that every CSV contains
    annotations and that the corresponding image file is contained in the same directory
    as the CSV.  The CSV can contain comments started with #, and must have the following
    fields: label, bbox upper left x, bbox upper left y, bbox width, bbox height,
    image file name, image width, image height.
    
    Arguments:
    
      anno_dir - Root of directory tree
      labels   - List of labels to include, default is all labels
      verbose  - If true display debugging output
      
    Return: A dictionary keyed on label where each value is a list of arrays containing
            examples of the corresponding label.
    """
    
    annos = defaultdict(list)
    
    # Walk the tree
    for root, subdir, files in os.walk(anno_dir):
        for file in files:
            
            # Process CSV files
            if not file.endswith('.csv'):
                continue
            csvpath = os.path.join(root, file)
            
            if verbose:
                print('Processing: %s' % csvpath)
                
            df = pd.read_csv(csvpath, comment = '#', names = ['label', 'bbox_x', 'bbox_y', 'bbox_width', 'bbox_height', 'image_file', 'image_width', 'image_height'])
            
            img_file = None
            for idx, row in df.iterrows():
                
                if verbose:
                    print('Annotation: %s' % row)
                    
                # Filter based on target labels
                if labels and row['label'] not in labels:
                    continue
                
                # Load the full image file
                if img_file is None or img_file != os.path.join(root, row['image_file']):
                    img_file = os.path.join(root, row['image_file'])
                    if not os.path.exists(img_file):
                        print('ERROR: Image file does not exist: %s' % img_file)
                        continue
                    img = Image.open(os.path.join(root, row['image_file']))
                    img = np.array(img)
                    
                    # print(img.shape)
                    
                    if len(img.shape) == 3 and img.shape[-1] > 3:
                        img = img[:,:,:3]
                        
                    if len(img.shape) == 2:

                        #ima.save(os.path.join(root, row['image_file']))

                        img = cv2.imread(os.path.join(root, row['image_file']))
                        img=np.array(img)
                        # if os.path.splitext(row['image_file'])[1] == ".jpg":
                        #    break

                    if verbose:
                        print('Loaded %s' % img_file)
                        print('Shape: %s' % str(img.shape))
                    
                # Extract the instance data
                ulx = row['bbox_x']
                uly = row['bbox_y']
                w = row['bbox_width']
                h = row['bbox_height']
                
                if ulx + w >= row['image_width'] or uly + h >= row['image_height']:
                    print('Error: bounding  box lies outside of image')
                    continue
                    
                if len(img.shape) == 3:
                    data = img[uly:uly+h,ulx:ulx+w,:]
                if len(img.shape) == 2:
                    data = img[uly:uly+h,ulx:ulx+w]
                print(data.shape)
                
                if verbose:
                    print(row['label'])
                    plt.imshow(data)
                    plt.show()
                
                annos[row['label']].append(data)
                
    return annos

In [4]:
# annos = load_annotations(verbose = True)
annos = load_annotations(verbose = False)

(46, 45, 3)
(42, 48, 3)
(45, 44, 3)
(42, 47, 3)
(42, 44, 3)
(44, 42, 3)
(44, 44, 3)
(41, 49, 3)
(48, 45, 3)
(42, 47, 3)
(44, 48, 3)
(46, 44, 3)
(24, 154, 3)
(50, 149, 3)
(25, 145, 3)
(14, 89, 3)
(16, 82, 3)
(14, 86, 3)
(15, 89, 3)
(17, 85, 3)
(17, 87, 3)
(25, 87, 3)
(21, 86, 3)
(24, 86, 3)
(26, 86, 3)
(22, 83, 3)
(38, 95, 3)
(33, 92, 3)
(30, 87, 3)
(30, 83, 3)
(34, 98, 3)
(34, 92, 3)
(35, 86, 3)
(60, 105, 3)
(67, 84, 3)
(33, 178, 3)
(66, 62, 3)
(52, 108, 3)
(54, 54, 3)
(57, 58, 3)
(56, 58, 3)
(54, 56, 3)
(235, 93, 3)
(183, 169, 3)
(166, 137, 3)
(169, 170, 3)
(151, 141, 3)
(166, 156, 3)
(135, 138, 3)
(126, 130, 3)
(126, 131, 3)
(127, 130, 3)
(127, 122, 3)
(128, 122, 3)
(126, 146, 3)
(123, 146, 3)
(130, 118, 3)
(131, 120, 3)
(87, 88, 3)
(86, 78, 3)
(111, 33, 3)
(68, 85, 3)
(74, 78, 3)
(38, 145, 3)
Error: bounding  box lies outside of image
(47, 123, 3)
(109, 43, 3)
Error: bounding  box lies outside of image
Error: bounding  box lies outside of image
(82, 27, 3)
(74, 45, 3)
(62, 54, 3)
(5

## Helper function to check if 2 Bounding Boxes overlap

In [5]:
class Point:
    def __init__(self, x, y, h, w):
        self.x = x
        self.y = y
        self.height = h
        self.width = w

def do_BB_Overlap(a, b):
    if (abs((a.x + a.width/2) - (b.x + b.width/2)) * 2 < (a.width + b.width)) and (abs((a.y + a.height/2) - (b.y + b.height/2)) * 2 < (a.height + b.height)):
        return True
    else:
        return False


# a = Point(10, 0, 5, 5)
# b = Point(10, 0, 5, 5)
# do_BB_Overlap(a, b)

# Function to test a location is empty or not

In [6]:
def test_location(x: int, y: int, height_k: int, width_k: int, overlap: list, img: np.ndarray, data_new: np.ndarray, df: pd.DataFrame, key: str):
    for coor_list in overlap:
        coordinates_overlap = [do_BB_Overlap(Point(x, y, height_k, width_k), Point(i[0], i[1], i[2], i[3])) for i in overlap]
        # True in coordinates_overlap indicates that bounding boxes overlap
        if True in coordinates_overlap:
            empty_location_found = False
            break
        else:
            empty_location_found = True
            break

    
    if empty_location_found:
        if verbose:
            print("img shape: "+ str(img[y:y+height_k, x:x+width_k, :].shape))
            print("data_new shape: " + str(data_new.shape))

        # adding the resized component array to full image array
        img[y:y+height_k, x:x+width_k, :] = data_new
        

        # Record location and type in the dataframe
        df = df.append({'label':key, 'ulx':x, 'uly':y, 'w':width_k, 'h':height_k}, ignore_index = True)
        # overlap.append([x, y, x+width_k, y+height_k])
        overlap.append([x, y, height_k, width_k])
        if verbose:
            print("Location inserted: (%d, %d)" % (x,y))
            # print(*overlap, sep = ", ")
    
    return df, overlap, empty_location_found
        

# generate drawing function new

In [7]:
def generate_drawings_new(annos, height, width, target, save_dir, image_name, verbose=False):
    """
    Generate a drawing.

    Parameters:

      annos  - Annotations obtained from load_annotations
      width  - Width of the drawing in pixels
      height - Height of the drawing in pixels

      target - Can be an integer, in which case that many random icons will be
               added to the drawing. Otherwise, it can be a dictionary with
               label keys and values that the are number of that type of icon
               to add to the drawing.
               For example: {'door':5, 'stairs':2} would create a drawing with
               5 doors and 2 stairs.
      save_dir - path where image should be saved
      

    Returns: The drawing as an array and a dataframe with information about
             what icons were used to create the drawing.
    """

    # Convert number of icons to dictionary format
    if type(target) == int:
        x = defaultdict(int)
        keys = list(annos.keys())
        for _ in range(target):
            x[random.choice(keys)] += 1
        target = x
        
    if verbose:
        # code diplaying the labels and their counts
        for key in target:
            print(key +": "+ str(target[key]))
    
    total_object_count = 0
    for key in target:
        total_object_count += target[key]
        
    # Initialize drawing
    img = np.zeros([width, height, 3], dtype=np.uint8)
    img.fill(255)
  
    # increment or decrement size while looking for available location in all the 8 directions
    gap_incr = 20


    # Add the desired number of icons for each type
    # create a dataframe to create the csv file for each image
    df = pd.DataFrame(columns=['label', 'ulx', 'uly', 'w', 'h'])

    # List to check for bounding box coordinate overlap
    overlap = []
    num_attempts = 5 # number of attempts to make for inserting object in the image

    
    for key in target:
        for _ in range(target[key]):
            if verbose:
                print("Object " + key + str(_)+" is added")
            
            # Choose random icon of the correct type
            data = random.choice(annos[key])

            # Converting into image for resizing
            img_re = Image.fromarray(data)

            # rotating our image for data augumentation
            rotation_angle = random.choice([0,90,180,270])
            rotated = img_re.rotate(rotation_angle, expand=1)
            #print("rotated shape ", rotated.size)

            # Place it in the drawing, w and h stands for width and height
            w = data.shape[1]
            h = data.shape[0]
            
            # width_k - new width for resizing
            # height_k - new height for resizing
            # w > h  - if placed horizontal
            # w < h  - if placed vertical
            width_k = None
            height_k = None

            # the sizes are calculated with 40 percentile to 60 percentile of width and height of object image
            if key.lower() == "door":
                if w > h: width_k = random.randint(109, 122)
                else: height_k = random.randint(36, 51)

            elif key.lower() == "window":
                if w > h: width_k = random.randint(83, 86)
                else: height_k = random.randint(29, 39)

            elif key.lower() == "bathtub":
                if w > h: width_k = random.randint(69, 99)
                else: height_k = random.randint(46, 63)

            elif key.lower() == "toilet":
                if w > h: width_k = random.randint(25, 28)
                else: height_k = random.randint(32, 33)

            elif key.lower() == "shower":
                if w > h: width_k = random.randint(59, 80)
                else: height_k = random.randint(54, 57)

            elif key.lower() == "sink":
                if w > h: width_k = random.randint(59, 64)
                else: height_k = random.randint(42, 46)

            elif key.lower() == "elevator":
                if w > h: width_k = random.randint(75, 111)
                else: height_k = random.randint(62, 82)

            elif key.lower() == "stairs":
                if w > h: width_k = random.randint(61, 84)
                else: height_k = random.randint(67, 87)

            wpercent = (w/float(h))
            if height_k == None:
                height_k = int(width_k/float(wpercent))

            if width_k == None:
                width_k = int(height_k * float(wpercent))

            # resizing our image
            resized = rotated.resize((width_k, height_k), Image.ANTIALIAS)
            # resized.show()
            # if verbose:
                # display(resized)
            
            # converting back into pixel array
            data_new = np.array(resized)
           
            
            attempts = 0
            empty_location_found = False

            while True:
                if (attempts == num_attempts):
                    if verbose:
                        print("Maximum attempts reached...")
                    # if empty location is not found till maximum random attempt, then go for sequential search
                    if not empty_location_found:
                        # nested for loop for sequential search in the image grid (width x height)
                        for x in range(0, (width - width_k), 20):
                            for y in range(0, (height - height_k), 20):
                                df, overlap, empty_location_found = test_location(x, y, height_k, width_k, overlap, img, data_new, df, key)
                                if empty_location_found:
                                    if verbose:
                                        print("empty location found with SERIAL Search")
                                    break # break from inner for loop
                            if empty_location_found:
                                break # break from outer for loop
                        # if the object can't be placed in the image despite of sequential search
                        if not empty_location_found:
                            if verbose:
                                print("No location found for the object")
                            break # break from while loop for the next object
                
                if empty_location_found:
                    if verbose:
                        print('Total attempts = %d' % (attempts))
                    break
                
                # x, y = random.choice(free_coordinates)
                # choose a random value from the image grid (width x height)
                rand_int = random.randint(0, width*height-1)
                # convert the grid value to corresponding (x, y) value
                x = rand_int % width
                y = rand_int // width
                
                       
                attempts += 1 # next attempt is made
                if x >= (width - width_k) or y >= (height - height_k):
                    continue # go for the next attempt
                    
                # Add any coordinates for first object in image file
                if len(overlap) == 0:
                    empty_location_found = True
                    # adding the resized component array to full image array
                    img[y:y+height_k, x:x+width_k, :] = data_new
                    # overlap_img[y:y+height_k, x:x+width_k, :] = 0

                    # Record location and type in the dataframe
                    df = df.append({'label':key, 'ulx':x, 'uly':y, 'w':width_k, 'h':height_k}, ignore_index = True)
                    overlap.append([x, y, height_k, width_k])
                    if verbose:
                        print("Location inserted: (%d, %d)" % (x,y))
                        print("First object is created")
                        # print(*overlap, sep = ", ")
                    
                    # break to for loop outside the while loop to insert next object
                    break 
                    
                # Compare current coordinates to other image coordinates
                for coor_list in overlap:
                    coordinates_overlap = [do_BB_Overlap(Point(x, y, height_k, width_k), Point(i[0], i[1], i[2], i[3])) for i in overlap]
                    # True in coordinates_overlap indicates that bounding boxes overlap
                    if True in coordinates_overlap:
                        empty_location_found = False
                        break
                    else:
                        empty_location_found = True
                        break
                    
                if empty_location_found:
                    # adding the resized component array to full image array
                    img[y:y+height_k, x:x+width_k, :] = data_new
                    

                    # Record location and type in the dataframe
                    df = df.append({'label':key, 'ulx':x, 'uly':y, 'w':width_k, 'h':height_k}, ignore_index = True)
                    # overlap.append([x, y, x+width_k, y+height_k])
                    overlap.append([x, y, height_k, width_k])
                    if verbose:
                        print("Location inserted: (%d, %d)" % (x,y))
                        # print(*overlap, sep = ", ")

                    # break to for loop outside the while loop to insert next object
                    break 
    
                    
                if not empty_location_found:
                    x_new_pos, x_new_neg = x, x
                    y_new_pos, y_new_neg = y, y
                                    
                    while (x_new_neg > 0 and x_new_pos < (width - width_k)) or (y_new_neg > 0 and y_new_pos < (height - height_k)):
                        # new coordinate positions are calculated with incrementing and decrementing the coordinate values
                        x_new_pos += gap_incr
                        y_new_pos += gap_incr
                        x_new_neg -= gap_incr
                        y_new_neg -= gap_incr                  
                        
                        # code for checking positive horizontal location
                        if x_new_pos < (width - width_k):
                            df, overlap, empty_location_found = test_location(x_new_pos, y, height_k, width_k, overlap, img, data_new, df, key)
                            if empty_location_found:
                                if verbose:
                                    print("horizontal location pos. and empty location found")
                                break
                        # code for checking positive vertical location
                        if y_new_pos < (height - height_k):
                            df, overlap, empty_location_found = test_location(x, y_new_pos, height_k, width_k, overlap, img, data_new, df, key)
                            if empty_location_found:
                                if verbose:
                                    print("vertical location pos. and empty location found")
                                break
                        # code for checking positive downward diagonal location
                        if x_new_pos < (width - width_k) and y_new_pos < (height - height_k):
                            df, overlap, empty_location_found = test_location(x_new_pos, y_new_pos, height_k, width_k, overlap, img, data_new, df, key)
                            if empty_location_found:
                                if verbose:
                                    print("diagonal location1 pos. and empty location found")
                                break
                        # code for checking positive upward diagonal location
                        if x_new_pos < (width - width_k) and y_new_neg > 0:
                            df, overlap, empty_location_found = test_location(x_new_pos, y_new_neg, height_k, width_k, overlap, img, data_new, df, key)
                            if empty_location_found:
                                if verbose:
                                    print("diagonal location2 pos. and empty location found")
                                break
                        # code for checking negative horizontal location
                        if x_new_neg > 0:
                            df, overlap, empty_location_found = test_location(x_new_neg, y, height_k, width_k, overlap, img, data_new, df, key)
                            if empty_location_found:
                                if verbose:
                                    print("horizontal location neg. and empty location found")
                                break
                        # code for checking negative vertical location
                        if y_new_neg > 0:
                            df, overlap, empty_location_found = test_location(x, y_new_neg, height_k, width_k, overlap, img, data_new, df, key)
                            if empty_location_found:
                                if verbose:
                                    print("vertical location neg. and empty location found")
                                break
                        # code for checking negative upward diagonal location
                        if x_new_neg > 0 and y_new_neg > 0:
                            df, overlap, empty_location_found = test_location(x_new_neg, y_new_neg, height_k, width_k, overlap, img, data_new, df, key)
                            if empty_location_found:
                                if verbose:
                                    print("diagonal location1 neg. and empty location found")
                                break
                        # code for checking negative downward diagonal location
                        if x_new_neg > 0 and y_new_pos < (height - height_k):
                            df, overlap, empty_location_found = test_location(x_new_neg, y_new_pos, height_k, width_k, overlap, img, data_new, df, key)
                            if empty_location_found:
                                if verbose:
                                    print("diagonal location2 neg. and empty location found")
                                break

                if empty_location_found:
                    break # break from while loop for the next object
                    # if not empty_location_found:
                    
    # if required number of objects are not there in the image then the image and its csv file rejected by returning None, None
    if len(df) != total_object_count:
        # print("len(df) = %d, total_object_count = %d None returned" % (len(df), total_object_count))
        return None

    extension = ".png"

    im = Image.fromarray(img)
    im.save(save_dir + image_name + extension)

    # saving labels
    df.to_csv(save_dir + image_name + '.csv')

    # plotting the image
    if verbose:
        plt.figure(figsize = (16,16)) 
        plt.imshow(img)
    
    # print("Return Success")
    return 1

# Code for execution of fake image generation

In [8]:
%%time
# The following code generates images with specified number of objects. If specifed number of objects is not generated then 
# the image is not generated. It can be figured out from the name of final image that how many images are not generated.

save_dir = "/home/sage08ai/Desktop/CV_tasks/door/output/"
os.makedirs(save_dir, exist_ok=True)
verbose = False

no_of_fake_images = 1000
image_trial_final = 5
# objects_in_image = 140
# objects_in_image = {'bathtub':2, 'door':25, 'inward_door':2, 'outward_door':2, 'elevator':12, 'shower':2, 'sink':5, 'stairs':18, 'toilet':26, 'window':6}
objects_in_image = {'bathtub':10, 'door':10, 'elevator':10, 'shower':10, 'sink':10, 'stairs':10, 'toilet':10, 'window':10}
image_counter = 0
image_name_counter = 0
image_trial_counter = 0

# for _ in range(no_of_fake_images):
while image_counter < no_of_fake_images:
    image_name = "fake_image_" + str(image_name_counter)
    # image_name = "fake_image_" + str(_)
    # generate_drawings(annos, 1000, 1000, random.randrange(10,100), save_dir, image_name, verbose=False)
    # To create an image with a list of given number of objects 
    # The list of annotations contain [bathtub: 11, door: 119, inward door: 12, outward door: 13, elevators: 64, shower: 9, 
    # sink: 28, stairs: 94, toilet: 124, window: 33]objects. So to generate images with 100 objects the following count of 
    # objects were considered and the input has to be given as a dictionary in the following way {bathtub: 2, door: 25, inward_door: 2, outward_door: 2, elevator: 12, shower: 1, sink: 5, 
    # stairs: 18, toilet: 26, window: 7}.
    # check_image_gen = generate_drawings(annos, 1000, 1000, {'bathtub':2, 'door':25, 'inward_door':2, 'outward_door':2, 'elevator':12, 'shower':2, 'sink':5, 'stairs':18, 'toilet':26, 'window':6}, save_dir, image_name, verbose = False)
    
    # if required number of objects are not created then it will return None else 1
    check_image_gen = generate_drawings_new(annos, 1000, 1000, objects_in_image, save_dir, image_name, verbose=False)
    
    # if return value is None then again an image is created with same name
    if check_image_gen == None: #when required number of objects couldn't be inserted
        image_trial_counter += 1
        if image_trial_counter != image_trial_final:
            # print("image trial counter = %d" % image_trial_counter)
            continue
        else:
            # print("image trial counter RESET")
            image_trial_counter = 0
    else: #if an image is returned from the function
        image_name_counter += 1
        if image_trial_counter != 0:
            image_trial_counter = 0
    image_counter += 1


CPU times: user 7min 17s, sys: 352 ms, total: 7min 17s
Wall time: 7min 17s


# To only view the image 

In [10]:
#generate_drawings_new(annos, 1000, 1000, 100, save_dir, image_name, verbose=True)

## Split data to train, val and test